In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Pyramid
def batch_pyramid(images):
    pyramid_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        G = img.copy()
        gp = [G]
        for i in range(4):
            G = cv2.pyrDown(G)
            gp.append(G)
        lp = [gp[4]]
        for i in range(4,0,-1):
            GE = cv2.pyrUp(gp[i])
            (x,y) = gp[i-1].shape
            L = cv2.subtract(gp[i-1],GE[:x,:y])
            lp.append(L)
        lp_1 = cv2.normalize(lp[1], lp[1],0,255,cv2.NORM_MINMAX)
        lp_2 = cv2.normalize(lp[2], lp[2],0,255,cv2.NORM_MINMAX)
        lp_out = [cv2.resize(lp_1, (128,128)),cv2.resize(lp_2, (128,128))]
        pyramid_list.append((np.array(lp_out)).astype(np.uint8))
    pyramid_list=np.array(pyramid_list,dtype = float32).reshape([images.shape[0],images.shape[1],images.shape[2],2])
    
    print (pyramid_list.shape)
    return pyramid_list

In [3]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../../image_gan/'
    TEST_DIR = '../../test/'

    # use this for full dataset
    train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images = train_images_gan
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "."
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [4]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

trainImage_pyramid = batch_pyramid(trainImage)
testImage_pyramid = batch_pyramid(testImage)
nb_classes = 2

# Convert class vectors to binary class matrices.
trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
print (trainLabels)
testLabels = keras.utils.to_categorical(testLabels, nb_classes)
print (testLabels)
print (testLabels.shape)

np.save('../trainImage.npy', trainImage)
np.save('../trainLabels.npy', trainLabels)
np.save('../testImage.npy', testImage)
np.save('../testLabels.npy', testLabels)
np.save('../trainImage_pyramid_L.npy', trainImage_pyramid)
np.save('../testImage_pyramid_L.npy', testImage_pyramid)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] trainImage_pyramid matrix: {:.2f}MB".format(
    (trainImage_pyramid.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_pyramid matrix: {:.4f}MB".format(
    (testImage_pyramid.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/6000
[INFO] processed 2000/6000
[INFO] processed 3000/6000
[INFO] processed 4000/6000
[INFO] processed 5000/6000
[INFO] processed 6000/6000
[INFO] processed 154/154
(6000, 128, 128, 3)


  6%|▋         | 10/154 [00:00<00:01, 99.94it/s]

(6000, 128, 128, 2)


100%|██████████| 154/154 [00:01<00:00, 97.91it/s] 


(154, 128, 128, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.pyramid_dim = 2  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_pyramid = np.load('../trainImage_pyramid_L.npy')
        self.test_x_pyramid = np.load('../testImage_pyramid_L.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_pyramid, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_pyramid",x_pyramid.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_pyramid,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_pyramid = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.pyramid_dim], 
                                name='x_pyramid')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_pyramid = self.train_x_pyramid[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_pyramid= shuffled_set_pyramid[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_pyramid: batch_x_pyramid,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_pyramid: batch_x_pyramid_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramidtest =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [6]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 15.24, total 50 epochs ptime: 780.21
#  [*] Training finished!
#  [*] Best Epoch:  5 , Accuracy:  0.8300000429153442
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-5
#  [*] Finished testing Best Epoch: 5 , accuracy:  0.8299999833106995 !


CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x2x32) [576, bytes: 2304]
cnn/co

Epoch: [ 2] [  40/  60] time: 28.0755, loss: 0.03550020
Epoch: [ 2] [  41/  60] time: 28.3157, loss: 0.06535789
Epoch: [ 2] [  42/  60] time: 28.5543, loss: 0.08034497
Epoch: [ 2] [  43/  60] time: 28.7937, loss: 0.04554632
Epoch: [ 2] [  44/  60] time: 29.0336, loss: 0.02392698
Epoch: [ 2] [  45/  60] time: 29.2747, loss: 0.02808484
Epoch: [ 2] [  46/  60] time: 29.5139, loss: 0.03479575
Epoch: [ 2] [  47/  60] time: 29.7534, loss: 0.06804153
Epoch: [ 2] [  48/  60] time: 29.9938, loss: 0.06866318
Epoch: [ 2] [  49/  60] time: 30.2334, loss: 0.02137995
Epoch: [ 2] [  50/  60] time: 30.4720, loss: 0.10215012
Epoch: [ 2] [  51/  60] time: 30.7103, loss: 0.05643531
Epoch: [ 2] [  52/  60] time: 30.9484, loss: 0.14355597
Epoch: [ 2] [  53/  60] time: 31.1884, loss: 0.11404132
Epoch: [ 2] [  54/  60] time: 31.4277, loss: 0.09005160
Epoch: [ 2] [  55/  60] time: 31.6675, loss: 0.07726948
Epoch: [ 2] [  56/  60] time: 31.9103, loss: 0.09834097
Epoch: [ 2] [  57/  60] time: 32.1502, loss: 0.0

Epoch: [ 5] [   3/  60] time: 65.8485, loss: 0.04784785
Epoch: [ 5] [   4/  60] time: 66.0891, loss: 0.04570676
Epoch: [ 5] [   5/  60] time: 66.3309, loss: 0.04472326
Epoch: [ 5] [   6/  60] time: 66.5709, loss: 0.02085410
Epoch: [ 5] [   7/  60] time: 66.8114, loss: 0.03195024
Epoch: [ 5] [   8/  60] time: 67.0533, loss: 0.01655407
Epoch: [ 5] [   9/  60] time: 67.2943, loss: 0.01892184
Epoch: [ 5] [  10/  60] time: 67.5351, loss: 0.02263657
Epoch: [ 5] [  11/  60] time: 67.7753, loss: 0.03548515
Epoch: [ 5] [  12/  60] time: 68.0185, loss: 0.03650708
Epoch: [ 5] [  13/  60] time: 68.2587, loss: 0.00645587
Epoch: [ 5] [  14/  60] time: 68.4992, loss: 0.03492861
Epoch: [ 5] [  15/  60] time: 68.7389, loss: 0.01452544
Epoch: [ 5] [  16/  60] time: 68.9813, loss: 0.02788367
Epoch: [ 5] [  17/  60] time: 69.2224, loss: 0.01292558
Epoch: [ 5] [  18/  60] time: 69.4625, loss: 0.04871840
Epoch: [ 5] [  19/  60] time: 69.7028, loss: 0.02963794
Epoch: [ 5] [  20/  60] time: 69.9466, loss: 0.0

Epoch: [ 7] [  27/  60] time: 102.9319, loss: 0.01621789
Epoch: [ 7] [  28/  60] time: 103.1724, loss: 0.00601315
Epoch: [ 7] [  29/  60] time: 103.4129, loss: 0.00355524
Epoch: [ 7] [  30/  60] time: 103.6534, loss: 0.00585790
Epoch: [ 7] [  31/  60] time: 103.8941, loss: 0.01393044
Epoch: [ 7] [  32/  60] time: 104.1361, loss: 0.00123485
Epoch: [ 7] [  33/  60] time: 104.3774, loss: 0.04125742
Epoch: [ 7] [  34/  60] time: 104.6174, loss: 0.00971221
Epoch: [ 7] [  35/  60] time: 104.8602, loss: 0.00558501
Epoch: [ 7] [  36/  60] time: 105.1016, loss: 0.00221418
Epoch: [ 7] [  37/  60] time: 105.3425, loss: 0.00162330
Epoch: [ 7] [  38/  60] time: 105.5818, loss: 0.02538374
Epoch: [ 7] [  39/  60] time: 105.8218, loss: 0.01274025
Epoch: [ 7] [  40/  60] time: 106.0633, loss: 0.00405847
Epoch: [ 7] [  41/  60] time: 106.3039, loss: 0.00027114
Epoch: [ 7] [  42/  60] time: 106.5443, loss: 0.00973268
Epoch: [ 7] [  43/  60] time: 106.7840, loss: 0.02972671
Epoch: [ 7] [  44/  60] time: 1

Epoch: [ 9] [  49/  60] time: 139.3125, loss: 0.01220352
Epoch: [ 9] [  50/  60] time: 139.5527, loss: 0.00092886
Epoch: [ 9] [  51/  60] time: 139.7934, loss: 0.00289079
Epoch: [ 9] [  52/  60] time: 140.0359, loss: 0.03294512
Epoch: [ 9] [  53/  60] time: 140.2768, loss: 0.00459965
Epoch: [ 9] [  54/  60] time: 140.5171, loss: 0.01701514
Epoch: [ 9] [  55/  60] time: 140.7567, loss: 0.00125308
Epoch: [ 9] [  56/  60] time: 140.9991, loss: 0.00047634
Epoch: [ 9] [  57/  60] time: 141.2399, loss: 0.00349137
Epoch: [ 9] [  58/  60] time: 141.4808, loss: 0.00352186
Epoch: [ 9] [  59/  60] time: 141.7216, loss: 0.00334559
[9/50] - ptime: 15.1979 loss: 0.01144099 acc: 0.57000 lr: 0.00100000
Epoch: [10] [   0/  60] time: 143.0626, loss: 0.00341646
Epoch: [10] [   1/  60] time: 143.3043, loss: 0.08223626
Epoch: [10] [   2/  60] time: 143.5461, loss: 0.00989520
Epoch: [10] [   3/  60] time: 143.7861, loss: 0.00714268
Epoch: [10] [   4/  60] time: 144.0277, loss: 0.00558350
Epoch: [10] [   5/ 

...

Epoch: [40] [  30/  60] time: 617.3287, loss: 0.00002012
Epoch: [40] [  31/  60] time: 617.5696, loss: 0.00002770
Epoch: [40] [  32/  60] time: 617.8096, loss: 0.00000393
Epoch: [40] [  33/  60] time: 618.0517, loss: 0.00004393
Epoch: [40] [  34/  60] time: 618.2925, loss: 0.00001859
Epoch: [40] [  35/  60] time: 618.5327, loss: 0.00001089
Epoch: [40] [  36/  60] time: 618.7728, loss: 0.00030442
Epoch: [40] [  37/  60] time: 619.0152, loss: 0.00003431
Epoch: [40] [  38/  60] time: 619.2565, loss: 0.00005339
Epoch: [40] [  39/  60] time: 619.4968, loss: 0.00017618
Epoch: [40] [  40/  60] time: 619.7371, loss: 0.00010419
Epoch: [40] [  41/  60] time: 619.9787, loss: 0.00015277
Epoch: [40] [  42/  60] time: 620.2199, loss: 0.00003717
Epoch: [40] [  43/  60] time: 620.4601, loss: 0.00000142
Epoch: [40] [  44/  60] time: 620.7005, loss: 0.00004090
Epoch: [40] [  45/  60] time: 620.9406, loss: 0.00004622
Epoch: [40] [  46/  60] time: 621.1816, loss: 0.00000242
Epoch: [40] [  47/  60] time: 6

Epoch: [42] [  52/  60] time: 653.7215, loss: 0.00001547
Epoch: [42] [  53/  60] time: 653.9622, loss: 0.00001406
Epoch: [42] [  54/  60] time: 654.2043, loss: 0.00006512
Epoch: [42] [  55/  60] time: 654.4446, loss: 0.00000228
Epoch: [42] [  56/  60] time: 654.6848, loss: 0.00014385
Epoch: [42] [  57/  60] time: 654.9247, loss: 0.00002870
Epoch: [42] [  58/  60] time: 655.1663, loss: 0.00004210
Epoch: [42] [  59/  60] time: 655.4074, loss: 0.00004742
[42/50] - ptime: 15.1701 loss: 0.00012429 acc: 0.72000 lr: 0.00065610
Epoch: [43] [   0/  60] time: 656.7329, loss: 0.00013818
Epoch: [43] [   1/  60] time: 656.9732, loss: 0.00002520
Epoch: [43] [   2/  60] time: 657.2148, loss: 0.00020889
Epoch: [43] [   3/  60] time: 657.4552, loss: 0.00019419
Epoch: [43] [   4/  60] time: 657.6951, loss: 0.00003996
Epoch: [43] [   5/  60] time: 657.9363, loss: 0.00000372
Epoch: [43] [   6/  60] time: 658.1795, loss: 0.00002453
Epoch: [43] [   7/  60] time: 658.4206, loss: 0.00002578
Epoch: [43] [   8/

Epoch: [45] [  13/  60] time: 690.9853, loss: 0.00001262
Epoch: [45] [  14/  60] time: 691.2238, loss: 0.00051780
Epoch: [45] [  15/  60] time: 691.4641, loss: 0.00024189
Epoch: [45] [  16/  60] time: 691.7070, loss: 0.00006145
Epoch: [45] [  17/  60] time: 691.9469, loss: 0.00076438
Epoch: [45] [  18/  60] time: 692.1886, loss: 0.00029862
Epoch: [45] [  19/  60] time: 692.4286, loss: 0.00000410
Epoch: [45] [  20/  60] time: 692.6688, loss: 0.00001214
Epoch: [45] [  21/  60] time: 692.9087, loss: 0.00008244
Epoch: [45] [  22/  60] time: 693.1506, loss: 0.00005076
Epoch: [45] [  23/  60] time: 693.3919, loss: 0.00002168
Epoch: [45] [  24/  60] time: 693.6317, loss: 0.00006806
Epoch: [45] [  25/  60] time: 693.8721, loss: 0.00030861
Epoch: [45] [  26/  60] time: 694.1139, loss: 0.00002733
Epoch: [45] [  27/  60] time: 694.3564, loss: 0.00034588
Epoch: [45] [  28/  60] time: 694.5967, loss: 0.00002484
Epoch: [45] [  29/  60] time: 694.8374, loss: 0.00000378
Epoch: [45] [  30/  60] time: 6

Epoch: [47] [  35/  60] time: 727.3790, loss: 0.00011535
Epoch: [47] [  36/  60] time: 727.6192, loss: 0.00015251
Epoch: [47] [  37/  60] time: 727.8603, loss: 0.00068058
Epoch: [47] [  38/  60] time: 728.1006, loss: 0.00081979
Epoch: [47] [  39/  60] time: 728.3421, loss: 0.00029023
Epoch: [47] [  40/  60] time: 728.5824, loss: 0.00099966
Epoch: [47] [  41/  60] time: 728.8226, loss: 0.00147346
Epoch: [47] [  42/  60] time: 729.0626, loss: 0.00334585
Epoch: [47] [  43/  60] time: 729.3043, loss: 0.00001752
Epoch: [47] [  44/  60] time: 729.5445, loss: 0.00027918
Epoch: [47] [  45/  60] time: 729.7848, loss: 0.00222420
Epoch: [47] [  46/  60] time: 730.0254, loss: 0.00271292
Epoch: [47] [  47/  60] time: 730.2699, loss: 0.00038012
Epoch: [47] [  48/  60] time: 730.5100, loss: 0.00097725
Epoch: [47] [  49/  60] time: 730.7497, loss: 0.00283007
Epoch: [47] [  50/  60] time: 730.9897, loss: 0.00061226
Epoch: [47] [  51/  60] time: 731.2324, loss: 0.00436847
Epoch: [47] [  52/  60] time: 7

Epoch: [49] [  57/  60] time: 763.7746, loss: 0.00001592
Epoch: [49] [  58/  60] time: 764.0144, loss: 0.00003859
Epoch: [49] [  59/  60] time: 764.2573, loss: 0.00004875
[49/50] - ptime: 15.1770 loss: 0.00019128 acc: 0.69000 lr: 0.00065610
Epoch: [50] [   0/  60] time: 765.5853, loss: 0.00017362
Epoch: [50] [   1/  60] time: 765.8266, loss: 0.00007521
Epoch: [50] [   2/  60] time: 766.0668, loss: 0.00008128
Epoch: [50] [   3/  60] time: 766.3110, loss: 0.00000710
Epoch: [50] [   4/  60] time: 766.5518, loss: 0.00000490
Epoch: [50] [   5/  60] time: 766.7929, loss: 0.00006600
Epoch: [50] [   6/  60] time: 767.0340, loss: 0.00313830
Epoch: [50] [   7/  60] time: 767.2732, loss: 0.00008894
Epoch: [50] [   8/  60] time: 767.5141, loss: 0.00001625
Epoch: [50] [   9/  60] time: 767.7552, loss: 0.00018540
Epoch: [50] [  10/  60] time: 767.9935, loss: 0.00005968
Epoch: [50] [  11/  60] time: 768.2350, loss: 0.00011146
Epoch: [50] [  12/  60] time: 768.4760, loss: 0.00000727
Epoch: [50] [  13/

In [7]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放